## Sentiment analysis <br> 

The objective of this problem is to perform Sentiment analysis from the tweets data collected from the users targeted at various mobile devices.
Based on the tweet posted by a user (text), we will classify if the sentiment of the user targeted at a particular mobile device is positive or not.

### 1. Read the dataset (tweets.csv) and drop the NA's while reading the dataset

In [1]:
#Import required standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
#Load the data from the local copy of the dataset into a pandas dataframe
#Also drop any rows with na values as it will not be useful
review_df = pd.read_csv("tweets.csv",encoding="unicode-escape").dropna()

In [19]:
#Sanity
review_df.sample(5)

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
8856,If there was ever any doubt on the influence o...,Apple,Positive emotion
1544,@mention @mention @mention &quot;@mention Goog...,Other Google product or service,Positive emotion
8683,I picked up a &quot;mophie&quot; iPhone charge...,Other Apple product or service,Positive emotion
5125,RT @mention @mention Google is looking at soci...,Google,Positive emotion
5020,"There's something 2 b said that here at #SXSW,...",iPad,Positive emotion


In [20]:
#Check the # of records and information of the dataframe
review_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3291 entries, 0 to 9088
Data columns (total 3 columns):
tweet_text                                            3291 non-null object
emotion_in_tweet_is_directed_at                       3291 non-null object
is_there_an_emotion_directed_at_a_brand_or_product    3291 non-null object
dtypes: object(3)
memory usage: 102.8+ KB


In [21]:
review_df.shape

(3291, 3)

### 2. Preprocess the text and add the preprocessed text in a column with name `text` in the dataframe.

In [22]:
def preprocess(text):
    try:
        return ''.join(i if ord(i)<128 else ' ' for i in text)
    except Exception as e:
        return ""

In [23]:
review_df['text'] = [preprocess(text) for text in review_df.tweet_text]

In [24]:
review_df.iloc[55][["tweet_text","text"]]

tweet_text    ÛÏ@mention &quot;Google before you tweet&quot...
text             @mention &quot;Google before you tweet&quot...
Name: 106, dtype: object

### 3. Consider only rows having a Positive or Negative emotion and remove other rows from the dataframe.

In [26]:
review_df.is_there_an_emotion_directed_at_a_brand_or_product.unique()

array(['Negative emotion', 'Positive emotion',
       'No emotion toward brand or product', "I can't tell"], dtype=object)

In [29]:
retained_emotions = ["Negative emotion","Positive emotion"]

review_df = review_df[review_df.is_there_an_emotion_directed_at_a_brand_or_product.isin(retained_emotions)]

In [30]:
#sanity
review_df.is_there_an_emotion_directed_at_a_brand_or_product.unique()

array(['Negative emotion', 'Positive emotion'], dtype=object)

In [33]:
#Retain only the text and emotions
re_df = review_df[["text","is_there_an_emotion_directed_at_a_brand_or_product"]]

In [36]:
re_df = re_df.rename(columns={"is_there_an_emotion_directed_at_a_brand_or_product" : "emotions"})

In [37]:
#Sanity
re_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3191 entries, 0 to 9088
Data columns (total 2 columns):
text        3191 non-null object
emotions    3191 non-null object
dtypes: object(2)
memory usage: 74.8+ KB


### 4. Represent text as numerical data using `CountVectorizer` and get the document term frequency matrix



In [31]:
#Import required modules for tokenization
from sklearn.feature_extraction.text import CountVectorizer

In [32]:
#Initialize the CV
cvect = CountVectorizer()

In [38]:
#Fit CV on text
cvect.fit(re_df.text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [41]:
#Transform the text
text_dtm = cvect.transform(re_df.text)

In [42]:
#Get the dtm now..it is a sparse matrix
print(text_dtm)

  (0, 79)	1
  (0, 225)	1
  (0, 415)	2
  (0, 1287)	1
  (0, 2282)	1
  (0, 2422)	1
  (0, 2637)	1
  (0, 2659)	1
  (0, 3300)	1
  (0, 3702)	1
  (0, 4141)	1
  (0, 4616)	1
  (0, 4769)	1
  (0, 5011)	1
  (0, 5141)	1
  (0, 5227)	1
  (0, 5370)	1
  (0, 5413)	1
  (1, 150)	1
  (1, 279)	1
  (1, 345)	1
  (1, 365)	1
  (1, 415)	1
  (1, 472)	1
  (1, 1348)	1
  :	:
  (3189, 284)	1
  (3189, 300)	2
  (3189, 345)	1
  (3189, 795)	1
  (3189, 797)	1
  (3189, 1813)	1
  (3189, 1933)	2
  (3189, 2271)	1
  (3189, 2482)	1
  (3189, 2627)	1
  (3189, 2637)	1
  (3189, 2659)	1
  (3189, 3205)	1
  (3189, 3276)	1
  (3189, 4199)	1
  (3189, 4592)	1
  (3189, 4707)	1
  (3189, 4769)	1
  (3189, 4781)	1
  (3189, 5244)	1
  (3189, 5274)	1
  (3190, 1696)	1
  (3190, 2627)	1
  (3190, 2905)	1
  (3190, 4769)	1


### 5. Find number of different words in vocabulary

In [43]:
#Check the vocabulary now
len(cvect.vocabulary_)

#There are 5610 words

5610

In [122]:
cvect.vocabulary_

{'wesley83': 5413,
 'have': 2282,
 '3g': 79,
 'iphone': 2637,
 'after': 225,
 'hrs': 2422,
 'tweeting': 5141,
 'at': 415,
 'rise_austin': 4141,
 'it': 2659,
 'was': 5370,
 'dead': 1287,
 'need': 3300,
 'to': 5011,
 'upgrade': 5227,
 'plugin': 3702,
 'stations': 4616,
 'sxsw': 4769,
 'jessedee': 2688,
 'know': 2779,
 'about': 150,
 'fludapp': 1904,
 'awesome': 472,
 'ipad': 2627,
 'app': 345,
 'that': 4915,
 'you': 5577,
 'll': 2923,
 'likely': 2893,
 'appreciate': 365,
 'for': 1933,
 'its': 2661,
 'design': 1348,
 'also': 279,
 'they': 4939,
 're': 3956,
 'giving': 2099,
 'free': 1965,
 'ts': 5110,
 'swonderlin': 4760,
 'can': 802,
 'not': 3361,
 'wait': 5342,
 'should': 4375,
 'sale': 4194,
 'them': 4925,
 'down': 1498,
 'hope': 2395,
 'this': 4953,
 'year': 5559,
 'festival': 1836,
 'isn': 2654,
 'as': 405,
 'crashy': 1180,
 'sxtxstate': 4794,
 'great': 2175,
 'stuff': 4681,
 'on': 3434,
 'fri': 1970,
 'marissa': 3059,
 'mayer': 3090,
 'google': 2138,
 'tim': 4986,
 'reilly': 4028,
 

In [131]:
cvect.get_feature_names()

#We can remove the digits etc

['000',
 '02',
 '03',
 '08',
 '10',
 '100',
 '100s',
 '100tc',
 '101',
 '106',
 '10am',
 '10k',
 '10mins',
 '10pm',
 '10x',
 '11',
 '11ntc',
 '11th',
 '12',
 '12b',
 '12th',
 '13',
 '130',
 '14',
 '1406',
 '1413',
 '1415',
 '15',
 '150',
 '1500',
 '150m',
 '157',
 '15am',
 '15k',
 '16162',
 '16gb',
 '16mins',
 '17',
 '188',
 '1986',
 '1990style',
 '1m',
 '1of',
 '1pm',
 '1st',
 '20',
 '200',
 '2010',
 '2011',
 '2012',
 '20s',
 '21',
 '22',
 '23',
 '24',
 '25',
 '250k',
 '25th',
 '2am',
 '2day',
 '2honor',
 '2moro',
 '2nd',
 '2nite',
 '2s',
 '2yrs',
 '30',
 '300',
 '3000',
 '30a',
 '30am',
 '30p',
 '30pm',
 '32',
 '32gb',
 '35',
 '36',
 '37',
 '3d',
 '3g',
 '3gs',
 '3k',
 '3rd',
 '3x',
 '40',
 '400',
 '40min',
 '41',
 '45',
 '45am',
 '47',
 '48',
 '4android',
 '4chan',
 '4g',
 '4nqv92l',
 '4sq',
 '4sq3',
 '4square',
 '50',
 '54',
 '55',
 '58',
 '59',
 '59p',
 '59pm',
 '5pm',
 '5th',
 '60',
 '64g',
 '64gb',
 '64gig',
 '64mb',
 '65',
 '6hours',
 '6th',
 '70',
 '75',
 '7th',
 '80',
 '800',

### 6. Find out how many Positive and Negative emotions are there.

Hint: Use value_counts on that column

In [44]:
#Get the positive and negative emotions
re_df.emotions.value_counts()

#More positive emotions

Positive emotion    2672
Negative emotion     519
Name: emotions, dtype: int64

In [49]:
#Check the null model accuracy..majority class
re_df.emotions.value_counts()[0]/re_df.emotions.size

0.8373550611093701

### 7. Change the labels for Positive and Negative emotions as 1 and 0 respectively and store in a different column in the same dataframe named 'Label'

Hint: use map on that column and give labels

In [57]:
#We will just map positive as 1 and negative as 0
re_df.emotions = re_df.emotions.map({"Positive emotion" : 1, "Negative emotion": 0})

In [58]:
#check sample
re_df.sample(5)

,text,emotions
3192,#sxsw Ze Frank I missed the Childhood Walk pro...,1
2913,What happened to the Taxi Magic iPhone app? No...,0
2038,A major #Apple iOS update the day before #SXSW...,0
8486,Nice! Austin Apple pop up shop in time for #SX...,1
3533,Great night of Interactive parties. And Congre...,1


In [59]:
re_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3191 entries, 0 to 9088
Data columns (total 2 columns):
text        3191 non-null object
emotions    3191 non-null int64
dtypes: int64(1), object(1)
memory usage: 74.8+ KB


### 8. Define the feature set (independent variable or X) to be `text` column and `labels` as target (or dependent variable)  and divide into train and test datasets

In [60]:
#Lets split the features and labels
X= re_df.text
y = re_df.emotions

In [61]:
#Sanity
X.shape

(3191,)

In [62]:
y.shape

(3191,)

## 9. **Predicting the sentiment:**


### Use Naive Bayes and Logistic Regression and print their accuracy scores for predicting the sentiment of the given text

In [77]:
#Import model libs
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
from sklearn.model_selection import train_test_split

In [66]:
#Lets first split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=5)

In [67]:
#Sanity
X_train.shape

(2552,)

In [68]:
X_test.shape

(639,)

In [91]:
#Use the cvect to transform the text
X_train_dtm = cvect.transform(X_train)
X_test_dtm = cvect.transform(X_test)

print('Features: ', X_train_dtm.shape[1])

Features:  5610


In [64]:
#initialize a MNB model and get the accuracy
mnb = MultinomialNB()

In [72]:
#Fit the model
mnb.fit(X_train_dtm,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [73]:
#predict
y_pred = mnb.predict(X_test_dtm)

In [78]:
#Check the accuracy and confusion matrix now
print("Accuracy Score",accuracy_score(y_test,y_pred))
print("Confusion Matrix\n",confusion_matrix(y_test,y_pred))
print("Classification Metrics\n",classification_report(y_test,y_pred))

#As probably expected the recall of 0 is very low
#The null model is only slightly less accurate

Accuracy Score 0.8497652582159625
Confusion Matrix
 [[ 45  68]
 [ 28 498]]
Classification Metrics
               precision    recall  f1-score   support

           0       0.62      0.40      0.48       113
           1       0.88      0.95      0.91       526

    accuracy                           0.85       639
   macro avg       0.75      0.67      0.70       639
weighted avg       0.83      0.85      0.84       639



In [79]:
#Lets try the Logistic Regression model
lr = LogisticRegression()

In [80]:
#Fit the model
lr.fit(X_train_dtm,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [83]:
#predict
y_pred = lr.predict(X_test_dtm)

In [84]:
#Check the accuracy and confusion matrix now
print("Accuracy Score",accuracy_score(y_test,y_pred))
print("Confusion Matrix\n",confusion_matrix(y_test,y_pred))
print("Classification Metrics\n",classification_report(y_test,y_pred))

#As probably expected the recall of 0 is very low
#Results are better than MNB

Accuracy Score 0.8841940532081377
Confusion Matrix
 [[ 48  65]
 [  9 517]]
Classification Metrics
               precision    recall  f1-score   support

           0       0.84      0.42      0.56       113
           1       0.89      0.98      0.93       526

    accuracy                           0.88       639
   macro avg       0.87      0.70      0.75       639
weighted avg       0.88      0.88      0.87       639



## 10. Create a function called `tokenize_test` which can take count vectorizer object as input, create document term matrix out of x_train & x_test, build and train a model using dtm created and print the accuracy 

In [102]:
def tokenize_test(vect):
    X_train_dtm = vect.fit_transform(X_train)
    print('Features: ', X_train_dtm.shape[1])
    X_test_dtm = vect.transform(X_test)
    print("Multinomial NB")
    nb = MultinomialNB()
    nb.fit(X_train_dtm, y_train)
    y_pred_class = nb.predict(X_test_dtm)
    print('Accuracy: ', accuracy_score(y_test, y_pred_class))
    print("Classification Metrics\n",classification_report(y_test, y_pred_class))
    print("Logistic Regression")
    lr = LogisticRegression()
    lr.fit(X_train_dtm, y_train)
    y_pred_class = lr.predict(X_test_dtm)
    print('Accuracy: ', accuracy_score(y_test, y_pred_class))
    print("Classification Metrics\n",classification_report(y_test, y_pred_class))

### Create a count vectorizer function which includes n_grams = 1,2  and pass it to tokenize_test function to print the accuracy score

In [103]:
cv = CountVectorizer(ngram_range=(1,2))

tokenize_test(cv)

#Features increased naturally significantly

#Accuracy actualy increased but recall for negative is drastically reduced for MNB
#LR is again better

Features:  25481
Multinomial NB
Accuracy:  0.86697965571205
Classification Metrics
               precision    recall  f1-score   support

           0       0.91      0.27      0.42       113
           1       0.86      0.99      0.92       526

    accuracy                           0.87       639
   macro avg       0.89      0.63      0.67       639
weighted avg       0.87      0.87      0.84       639

Logistic Regression
Accuracy:  0.8982785602503912
Classification Metrics
               precision    recall  f1-score   support

           0       0.93      0.46      0.62       113
           1       0.90      0.99      0.94       526

    accuracy                           0.90       639
   macro avg       0.91      0.73      0.78       639
weighted avg       0.90      0.90      0.88       639



### Create a count vectorizer function with stopwords = 'english'  and pass it to tokenize_test function to print the accuracy score

In [104]:
cv = CountVectorizer(stop_words="english")

tokenize_test(cv)

#Feature size reduced
#Accuracy actualy increased slightly from the base model but recall is again very bad for MNB
#For LR the accuracy decreased and recall is also very bad

Features:  4775
Multinomial NB
Accuracy:  0.8575899843505478
Classification Metrics
               precision    recall  f1-score   support

           0       0.79      0.27      0.40       113
           1       0.86      0.98      0.92       526

    accuracy                           0.86       639
   macro avg       0.83      0.63      0.66       639
weighted avg       0.85      0.86      0.83       639

Logistic Regression
Accuracy:  0.8732394366197183
Classification Metrics
               precision    recall  f1-score   support

           0       0.83      0.35      0.50       113
           1       0.88      0.98      0.93       526

    accuracy                           0.87       639
   macro avg       0.85      0.67      0.71       639
weighted avg       0.87      0.87      0.85       639



### Create a count vectorizer function with stopwords = 'english' and max_features =300  and pass it to tokenize_test function to print the accuracy score

In [105]:
cv = CountVectorizer(stop_words="english",max_features=300)

tokenize_test(cv)

#Accuracy actualy decreased but recall seems to be slightly better for MNB
#For LR accuracy reduced and recall also reduced

Features:  300
Multinomial NB
Accuracy:  0.8247261345852895
Classification Metrics
               precision    recall  f1-score   support

           0       0.51      0.35      0.41       113
           1       0.87      0.93      0.90       526

    accuracy                           0.82       639
   macro avg       0.69      0.64      0.65       639
weighted avg       0.80      0.82      0.81       639

Logistic Regression
Accuracy:  0.8482003129890454
Classification Metrics
               precision    recall  f1-score   support

           0       0.71      0.24      0.36       113
           1       0.86      0.98      0.91       526

    accuracy                           0.85       639
   macro avg       0.78      0.61      0.64       639
weighted avg       0.83      0.85      0.82       639



### Create a count vectorizer function with n_grams = 1,2  and max_features = 15000  and pass it to tokenize_test function to print the accuracy score

In [106]:
cv = CountVectorizer(ngram_range=(1,2),max_features=15000)

tokenize_test(cv)

#Accuracy actualy increased slightly and recall seems to be slightly better
#For LR model also accuracy and recall increased

Features:  15000
Multinomial NB
Accuracy:  0.8685446009389671
Classification Metrics
               precision    recall  f1-score   support

           0       0.84      0.32      0.46       113
           1       0.87      0.99      0.93       526

    accuracy                           0.87       639
   macro avg       0.85      0.65      0.69       639
weighted avg       0.86      0.87      0.84       639

Logistic Regression
Accuracy:  0.8951486697965572
Classification Metrics
               precision    recall  f1-score   support

           0       0.88      0.47      0.61       113
           1       0.90      0.99      0.94       526

    accuracy                           0.90       639
   macro avg       0.89      0.73      0.78       639
weighted avg       0.89      0.90      0.88       639



### Create a count vectorizer function with n_grams = 1,2  and include terms that appear at least 2 times (min_df = 2)  and pass it to tokenize_test function to print the accuracy score

In [107]:
cv = CountVectorizer(ngram_range=(1,2),min_df=2)

tokenize_test(cv)

#Accuracy actualy increased slightly and recall seems to be slightly better
#Even for LR model accuracy and recall is better

Features:  8370
Multinomial NB
Accuracy:  0.8701095461658842
Classification Metrics
               precision    recall  f1-score   support

           0       0.75      0.40      0.52       113
           1       0.88      0.97      0.92       526

    accuracy                           0.87       639
   macro avg       0.82      0.68      0.72       639
weighted avg       0.86      0.87      0.85       639

Logistic Regression
Accuracy:  0.8967136150234741
Classification Metrics
               precision    recall  f1-score   support

           0       0.87      0.49      0.62       113
           1       0.90      0.98      0.94       526

    accuracy                           0.90       639
   macro avg       0.89      0.74      0.78       639
weighted avg       0.89      0.90      0.88       639



## All parameters used along ngram of (1,2)

In [115]:
cv = CountVectorizer(ngram_range=(1,2),min_df=2,max_features=5000,stop_words="english",max_df=0.8)

tokenize_test(cv)

#Accuracy actualy increased slightly and recall seems to be slightly better
#Even for LR model accuracy and recall is better

Features:  5000
Multinomial NB
Accuracy:  0.8403755868544601
Classification Metrics
               precision    recall  f1-score   support

           0       0.58      0.35      0.43       113
           1       0.87      0.95      0.91       526

    accuracy                           0.84       639
   macro avg       0.73      0.65      0.67       639
weighted avg       0.82      0.84      0.82       639

Logistic Regression
Accuracy:  0.8779342723004695
Classification Metrics
               precision    recall  f1-score   support

           0       0.83      0.39      0.53       113
           1       0.88      0.98      0.93       526

    accuracy                           0.88       639
   macro avg       0.86      0.69      0.73       639
weighted avg       0.87      0.88      0.86       639



## Lets try with TF-IDF vectorizer

In [116]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [118]:
tvect = TfidfVectorizer()

tokenize_test(tvect)

Features:  5013
Multinomial NB
Accuracy:  0.8278560250391236
Classification Metrics
               precision    recall  f1-score   support

           0       1.00      0.03      0.05       113
           1       0.83      1.00      0.91       526

    accuracy                           0.83       639
   macro avg       0.91      0.51      0.48       639
weighted avg       0.86      0.83      0.75       639

Logistic Regression
Accuracy:  0.8419405320813772
Classification Metrics
               precision    recall  f1-score   support

           0       1.00      0.11      0.19       113
           1       0.84      1.00      0.91       526

    accuracy                           0.84       639
   macro avg       0.92      0.55      0.55       639
weighted avg       0.87      0.84      0.79       639



In [125]:
tvect = TfidfVectorizer(ngram_range=(1,2),min_df=2,stop_words="english",max_features=3000)

tokenize_test(tvect)

Features:  3000
Multinomial NB
Accuracy:  0.8450704225352113
Classification Metrics
               precision    recall  f1-score   support

           0       0.94      0.13      0.23       113
           1       0.84      1.00      0.91       526

    accuracy                           0.85       639
   macro avg       0.89      0.57      0.57       639
weighted avg       0.86      0.85      0.79       639

Logistic Regression
Accuracy:  0.837245696400626
Classification Metrics
               precision    recall  f1-score   support

           0       1.00      0.08      0.15       113
           1       0.83      1.00      0.91       526

    accuracy                           0.84       639
   macro avg       0.92      0.54      0.53       639
weighted avg       0.86      0.84      0.78       639



In [126]:
##TF-IDF model does not help in this case...the recall is terrible and the model accuracy is also not great

## Lets try a deep learning model with CountVectorizer and see what happens

In [127]:
import tensorflow as tf

In [128]:
tf.keras.backend.clear_session()
model = tf.keras.Sequential()

In [129]:
#Add hidden layers
model.add(tf.keras.layers.Dense(100, activation='relu', input_shape=(len(cvect.vocabulary_),)))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))

#Add Output layer
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [130]:
#Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [132]:
model.fit(X_train_dtm, y_train,
           validation_data=(X_test_dtm, y_test), 
           epochs=10, batch_size=32)

Train on 2552 samples, validate on 639 samples
Epoch 1/10
2552/2552 [==============================] - 1s 333us/sample - loss: 0.4553 - acc: 0.8393 - val_loss: 0.3993 - val_acc: 0.8232
Epoch 2/10
2552/2552 [==============================] - 1s 246us/sample - loss: 0.3143 - acc: 0.8405 - val_loss: 0.3534 - val_acc: 0.8451
Epoch 3/10
2552/2552 [==============================] - 1s 245us/sample - loss: 0.2187 - acc: 0.8981 - val_loss: 0.3285 - val_acc: 0.8701
Epoch 4/10
2552/2552 [==============================] - 1s 233us/sample - loss: 0.1229 - acc: 0.9600 - val_loss: 0.3781 - val_acc: 0.8717
Epoch 5/10
2552/2552 [==============================] - 1s 229us/sample - loss: 0.0693 - acc: 0.9777 - val_loss: 0.4417 - val_acc: 0.8732
Epoch 6/10
2552/2552 [==============================] - 1s 247us/sample - loss: 0.0437 - acc: 0.9882 - val_loss: 0.4845 - val_acc: 0.8701
Epoch 7/10
2552/2552 [==============================] - 1s 240us/sample - loss: 0.0320 - acc: 0.9929 - val_loss: 0.5300 - val

In [133]:
y_pred_class = model.predict_classes(X_test_dtm)

In [134]:
#Check the accuracy and confusion matrix now
print("Accuracy Score",accuracy_score(y_test,y_pred_class))
print("Confusion Matrix\n",confusion_matrix(y_test,y_pred_class))
print("Classification Metrics\n",classification_report(y_test,y_pred_class))

Accuracy Score 0.8685446009389671
Confusion Matrix
 [[ 56  57]
 [ 27 499]]
Classification Metrics
               precision    recall  f1-score   support

           0       0.67      0.50      0.57       113
           1       0.90      0.95      0.92       526

    accuracy                           0.87       639
   macro avg       0.79      0.72      0.75       639
weighted avg       0.86      0.87      0.86       639

